# Teste de engenharia de dados Banco ABC
Este notebook apresenta um exemplo de como transformar seus dados armazenados "in-memory" em tabelas no SQLite, permitindo a manipulação através de querys SQL. Caso ainda tenha dúvidas, você pode utilizar tutoriais [como esse](https://deepnote.com/blog/query-pandas-dataframes-with-sql) para ver mais detalhes. Esse snippet foi testado utilizando o Python 3.10.

O primeiro passo é importar as bibliotecas necessárias. Caso você não as tenha instalada na máquina, será necessário realizar isso antes.

In [3]:
import pandas as pd
import sqlalchemy
%load_ext sql

Feito isso, você deve fazer a leitura das bases de dados para a memória. O exemplo abaixo é um pseudocódigo.

In [4]:
clients = pd.read_csv('data/clientes.csv')

Depois, você irá inicializar a SQLite através do código abaixo:

In [5]:
engine = sqlalchemy.create_engine('sqlite:///mydatabase.db')
%sql sqlite:///mydatabase.db

Agora, basta transformar seus dados em tabelas efetivamente. É possível que você se depare com o seguinte erro: **ValueError: Unsigned 64 bit integer datatype is not supported**. Isso ocorre por que o SQLite não oferece suporte para o tipo **uint64**. Nesse caso, você deve tentar converter as colunas do seu dataset que estão em uint64 para string. As colunas que recebem esse tipo geralmente são *numeroCartao* e *numeroConta*.

In [6]:
clients.to_sql('clientes', con=engine, if_exists='replace')

250000

Finalmente, você deve ser capaz de executar querys utilizando a seguinte syntax:

In [7]:
query = %sql """ SELECT * \
                 FROM clientes limit 10"""

 * sqlite:///mydatabase.db
Done.


In [8]:
print(query)

+-------+-------------+------------------------+----------------+-----------------------------+------------------+--------------+
| index |     cpf     |          nome          | dataNascimento |            email            |       city       | dataCadastro |
+-------+-------------+------------------------+----------------+-----------------------------+------------------+--------------+
|   0   | 18046079841 |     Beatriz Porto      |   1988-01-13   |     ocorreia@cardoso.org    |     Ferreira     |  2012-06-05  |
|   1   | 81465703213 |  Luiz Miguel Carvalho  |   1996-10-02   |    theoviana@hotmail.com    |   Dias Grande    |  2011-11-27  |
|   2   | 58695342240 |      Calebe Sales      |   2003-09-28   |   luiz-otavioda-cruz@da.br  |   Costa Grande   |  2020-01-23  |
|   3   | 78857172116 |  Srta. Bárbara Duarte  |   1968-11-19   | silveiraisadora@martins.com | Ferreira de Melo |  2021-07-09  |
|   4   | 80101594884 |  Francisco Nascimento  |   1988-08-08   |      bda-luz@gmail.com  

## Primeira fase - Query 1
Extraia o nome, CPF e e-mail de todos os clientes que tenham ao menos R$ 400,00 reais de compras aprovadas nos últimos dois meses. Além disso, esses clientes precisam estar com a conta ativa e com o cartão desbloqueado, a menos que o código de bloqueio do cartão seja igual a “M”. 

In [9]:
# Tentar diferentes codificações
try:
    transactions = pd.read_csv('data/transacoes.csv', encoding='utf-8')
except UnicodeDecodeError:
    transactions = pd.read_csv('data/transacoes.csv', encoding='latin1')

In [10]:
# Carregar os DataFrames com o formato correto
accounts = pd.read_csv('data/contas.csv', delimiter=';')
cards = pd.read_excel('data/cartoes.xlsx')  # Ler o arquivo Excel
#transactions = pd.read_csv('data/transacoes.csv', delimiter=';')

In [11]:
# Converter colunas uint64 para int64
accounts['numeroConta'] = accounts['numeroConta'].astype('int64')
cards['numeroCartao'] = cards['numeroCartao'].astype('int64')
transactions['numeroCartao'] = transactions['numeroCartao'].astype('int64')

In [12]:
# Verificar os tipos de dados das colunas
print("Tipos de dados em accounts:", accounts.dtypes)
print("Tipos de dados em cards:", cards.dtypes)
print("Tipos de dados em transactions:", transactions.dtypes)


Tipos de dados em accounts: codMatricula    object
cpfCliente       int64
numeroConta      int64
agencia          int64
dataAbertura    object
ativo            int64
saldo            int64
dtype: object
Tipos de dados em cards: codMatricula                 object
numeroCartao                  int64
cartaoBloqueado                bool
codBloqueio                  object
nomeImprCart                 object
digitoVerificador             int64
dataEmisCartao       datetime64[ns]
limiteTotal                 float64
limiteRestante              float64
dtype: object
Tipos de dados em transactions: numeroCartao      int64
dataCompra       object
aprovado          int64
idLoja            int64
qtdParcelas       int64
valorCompra     float64
nomeLoja         object
dtype: object


In [13]:
# Inserir DataFrames no banco de dados
accounts.to_sql('contas', con=engine, if_exists='replace', index=False)
cards.to_sql('cartoes', con=engine, if_exists='replace', index=False)
transactions.to_sql('transacoes', con=engine, if_exists='replace', index=False)

1250000

In [14]:
# Obter a data mais recente de dataCompra
query_data_recente = %sql SELECT MAX(dataCompra) AS dataRecente FROM transacoes;

# Obter a data subtraída de dois meses
query_data_subtrair_dois_meses = %sql SELECT DATE(MAX(dataCompra), '-2 months') AS dataMenosDoisMeses FROM transacoes;

 * sqlite:///mydatabase.db
Done.
 * sqlite:///mydatabase.db
Done.


In [15]:
print(f'A data de compra mais recente da base é: {query_data_recente}')
print('')
print(f'A data de compra de dois meses para trás é: {query_data_subtrair_dois_meses}')

A data de compra mais recente da base é: +-------------+
| dataRecente |
+-------------+
|  2024-02-21 |
+-------------+

A data de compra de dois meses para trás é: +--------------------+
| dataMenosDoisMeses |
+--------------------+
|     2023-12-21     |
+--------------------+


In [16]:
query_1 = %sql SELECT clientes.nome, clientes.cpf, clientes.email FROM clientes JOIN contas ON clientes.cpf = contas.cpfCliente JOIN cartoes ON contas.codMatricula = cartoes.codMatricula JOIN transacoes ON cartoes.numeroCartao = transacoes.numeroCartao WHERE contas.ativo = 1 AND (cartoes.cartaoBloqueado = 0 OR cartoes.codBloqueio = 'M') AND transacoes.aprovado = 1 AND transacoes.dataCompra BETWEEN '2024-02-21' AND '2024-12-21' GROUP BY clientes.nome, clientes.cpf, clientes.email HAVING SUM(transacoes.valorCompra) >= 400 limit 10;
print(query_1)

 * sqlite:///mydatabase.db
Done.
+--------------------+-------------+--------------------------+
|        nome        |     cpf     |          email           |
+--------------------+-------------+--------------------------+
|    Agatha Gomes    | 47030856073 |  larissa95@yahoo.com.br  |
| Agatha Nascimento  | 53686156562 | fogacaraquel@silveira.br |
|    Agatha Nunes    | 36537807266 |     gmelo@freitas.br     |
|  Agatha Oliveira   | 65631925523 |    yasmin76@barros.br    |
|   Agatha Ribeiro   | 71612587231 |  matheus75@monteiro.org  |
|    Alana Farias    | 49928102324 |   wmartins@aragao.com    |
|   Alana Martins    | 92950456450 |   ybarros@hotmail.com    |
|   Alana Monteiro   | 45154499126 |   catarina06@ig.com.br   |
|    Alana Viana     | 87403196818 |   mariana74@uol.com.br   |
| Alana da Conceição | 11433350236 |  cunhagiovanna@alves.br  |
+--------------------+-------------+--------------------------+


In [17]:
#Testes de consulta realizados
teste = %sql SELECT clientes.nome, clientes.cpf, clientes.email FROM clientes LIMIT 10
print(teste)

 * sqlite:///mydatabase.db
Done.
+------------------------+-------------+-----------------------------+
|          nome          |     cpf     |            email            |
+------------------------+-------------+-----------------------------+
|     Beatriz Porto      | 18046079841 |     ocorreia@cardoso.org    |
|  Luiz Miguel Carvalho  | 81465703213 |    theoviana@hotmail.com    |
|      Calebe Sales      | 58695342240 |   luiz-otavioda-cruz@da.br  |
|  Srta. Bárbara Duarte  | 78857172116 | silveiraisadora@martins.com |
|  Francisco Nascimento  | 80101594884 |      bda-luz@gmail.com      |
|      Laura da Luz      | 12215420147 |      yasmin37@gmail.com     |
|  Ana Carolina Freitas  | 35010549464 |  danielda-mota@yahoo.com.br |
| Maria Fernanda Pereira | 70452017421 |    lunafogaca@hotmail.com   |
|     Bryan da Rocha     | 31815402513 |      ssales@uol.com.br      |
|     Renan Silveira     | 14897402109 |       nicole06@da.org       |
+------------------------+-------------+----

In [18]:
# Definindo a consulta SQL
query_1 = """
SELECT clientes.nome, clientes.cpf, clientes.email 
FROM clientes 
JOIN contas ON clientes.cpf = contas.cpfCliente 
JOIN cartoes ON contas.codMatricula = cartoes.codMatricula 
JOIN transacoes ON cartoes.numeroCartao = transacoes.numeroCartao 
WHERE contas.ativo = 1 
AND (cartoes.cartaoBloqueado = 0 OR cartoes.codBloqueio = 'M') 
AND transacoes.aprovado = 1 
AND transacoes.dataCompra BETWEEN '2024-02-21' AND '2024-12-21' 
GROUP BY clientes.nome, clientes.cpf, clientes.email 
HAVING SUM(transacoes.valorCompra) >= 400;
"""

In [19]:
# Executando a consulta e carregando o resultado em um DataFrame
df1 = pd.read_sql(query_1, engine)

# Slavando o DataFrame em um arquivo CSV
df1.to_csv('output/resultado_query.csv', index=False)

### Fase um query_2 - Ranking

In [24]:
import pandas as pd
from sqlalchemy import create_engine

# Conexão com o banco de dados SQLite
engine = create_engine('sqlite:///mydatabase.db')

# Query ajustada
query_ranking = """
WITH Compradores AS (
    SELECT 
        c.cpf,
        a.numeroConta,
        t.numeroCartao,
        AVG(tr.valorCompra) AS media_compras_ultimos_6_meses,
        SUM(tr.valorCompra) / t.limiteTotal AS percentual_limite_utilizado
    FROM clientes c
    JOIN contas a ON c.cpf = a.cpfCliente
    JOIN cartoes t ON a.codMatricula = t.codMatricula
    JOIN transacoes tr ON t.numeroCartao = tr.numeroCartao
    WHERE tr.dataCompra BETWEEN DATE('2024-02-21', '-6 months') AND '2024-02-21'
      AND tr.aprovado = 1
      AND tr.idLoja IN (6, 18, 24, 25)
    GROUP BY c.cpf, a.numeroConta, t.numeroCartao
    HAVING AVG(tr.valorCompra) >= 300
       AND SUM(tr.valorCompra) / t.limiteTotal >= 0.70
),
Nao_Compradores AS (
    SELECT 
        c.cpf,
        a.numeroConta,
        t.numeroCartao,
        SUM(tr.valorCompra) / t.limiteTotal AS percentual_limite_utilizado,
        a.saldo AS saldo_conta
    FROM clientes c
    JOIN contas a ON c.cpf = a.cpfCliente
    JOIN cartoes t ON a.codMatricula = t.codMatricula
    LEFT JOIN transacoes tr ON t.numeroCartao = tr.numeroCartao
    WHERE a.ativo = 1
      AND (t.cartaoBloqueado = 0 OR t.codBloqueio = 'M')
    GROUP BY c.cpf, a.numeroConta, t.numeroCartao, a.saldo
),
Ranking_Compradores AS (
    SELECT
        cpf,
        numeroConta,
        numeroCartao,
        'A' AS ranking
    FROM Compradores
),
Ranking_Nao_Compradores AS (
    SELECT
        cpf,
        numeroConta,
        numeroCartao,
        CASE 
            WHEN percentual_limite_utilizado BETWEEN 0.70 AND 0.80 AND saldo_conta > 10000 THEN 'B'
            WHEN percentual_limite_utilizado BETWEEN 0.80 AND 0.90 AND saldo_conta > 15000 THEN 'C'
            WHEN percentual_limite_utilizado BETWEEN 0.90 AND 0.95 AND saldo_conta > 20000 THEN 'D'
            ELSE ''
        END AS ranking
    FROM Nao_Compradores
)
SELECT
    c.cpf,
    a.numeroConta,
    t.numeroCartao,
    COALESCE(rc.ranking, rnc.ranking, '') AS ranking
FROM clientes c
JOIN contas a ON c.cpf = a.cpfCliente
JOIN cartoes t ON a.codMatricula = t.codMatricula
LEFT JOIN Ranking_Compradores rc ON rc.cpf = c.cpf AND rc.numeroConta = a.numeroConta AND rc.numeroCartao = t.numeroCartao
LEFT JOIN Ranking_Nao_Compradores rnc ON rnc.cpf = c.cpf AND rnc.numeroConta = a.numeroConta AND rnc.numeroCartao = t.numeroCartao
WHERE a.ativo = 1
AND (t.cartaoBloqueado = 0 OR t.codBloqueio = 'M')
"""

# Executar a query e carregar o resultado em um DataFrame
df_ranking = pd.read_sql(query_ranking, engine)

# Exibir o DataFrame com os resultados
print(df_ranking)


                cpf          numeroConta         numeroCartao ranking
0       18046079841  5100505060838687527  2097828104877643350        
1       81465703213  7595295926917893421  3442201103375110717       A
2       58695342240  1762048877978329900  4818239517605922667        
3       78857172116  9117145463574074166  1626022023864896110        
4       80101594884  4890295503896255487  1801738842007609011        
...             ...                  ...                  ...     ...
140446  27755193741  3684920847235535896  1573015684869023228        
140447  65263158321  3070571305655218977  3373258549827659228        
140448  97759347157  4029730621197334551  2356024123535096242        
140449  72734262475  7647977361733954739  6938887299867480048        
140450  47849459095  4429967981554532148  5316962919948107269        

[140451 rows x 4 columns]


### Testando Conexão com o banco de dados

In [26]:
#Teste de conexão
from sqlalchemy import create_engine

# Criar conexão com o banco de dados existente
engine = create_engine('sqlite:///mydatabase.db')

# Verificar se a conexão foi estabelecida
try:
    with engine.connect() as conn:
        print("Conexão bem-sucedida com o banco de dados")
except Exception as e:
    print(f"Erro ao conectar com o banco de dados: {e}")


Conexão bem-sucedida com o banco de dados


In [27]:
# Verificar se há dados na tabela 'clientes'
clientes = pd.read_sql("SELECT * FROM clientes LIMIT 5;", engine)
print(clientes)

# Verificar se há dados na tabela 'contas'
contas = pd.read_sql("SELECT * FROM contas LIMIT 5;", engine)
print(contas)


   index          cpf                  nome dataNascimento  \
0      0  18046079841         Beatriz Porto     1988-01-13   
1      1  81465703213  Luiz Miguel Carvalho     1996-10-02   
2      2  58695342240          Calebe Sales     2003-09-28   
3      3  78857172116  Srta. Bárbara Duarte     1968-11-19   
4      4  80101594884  Francisco Nascimento     1988-08-08   

                         email              city dataCadastro  
0         ocorreia@cardoso.org          Ferreira   2012-06-05  
1        theoviana@hotmail.com       Dias Grande   2011-11-27  
2     luiz-otavioda-cruz@da.br      Costa Grande   2020-01-23  
3  silveiraisadora@martins.com  Ferreira de Melo   2021-07-09  
4            bda-luz@gmail.com              Dias   2015-10-25  
                           codMatricula   cpfCliente          numeroConta  \
0  0617cde1-63d6-477a-9015-4a206f1f9d81  18046079841  5100505060838687527   
1  7703a577-b7f3-4543-a952-7baa88537e4d  81465703213  7595295926917893421   
2  39d4e63c-

In [29]:
# Executar a query de ranking
result = pd.read_sql(query_ranking, engine)

# Exibir o resultado
print(result)

                cpf          numeroConta         numeroCartao ranking
0       18046079841  5100505060838687527  2097828104877643350        
1       81465703213  7595295926917893421  3442201103375110717       A
2       58695342240  1762048877978329900  4818239517605922667        
3       78857172116  9117145463574074166  1626022023864896110        
4       80101594884  4890295503896255487  1801738842007609011        
...             ...                  ...                  ...     ...
140446  27755193741  3684920847235535896  1573015684869023228        
140447  65263158321  3070571305655218977  3373258549827659228        
140448  97759347157  4029730621197334551  2356024123535096242        
140449  72734262475  7647977361733954739  6938887299867480048        
140450  47849459095  4429967981554532148  5316962919948107269        

[140451 rows x 4 columns]


In [169]:
# Exportar o DataFrame para o diretório 'output'
result.to_csv('output/ranking_clientes.csv', index=False)

Verificando data types das colunas para fazer transformação no ETL correta

In [33]:
print(df1.dtypes)


nome     object
cpf       int64
email    object
dtype: object


In [34]:
print(result.dtypes)

cpf              int64
numeroConta      int64
numeroCartao     int64
ranking         object
dtype: object


In [43]:
# Lendo aqreuivo parquet 1.
file_path = 'output/resultado_query.parquet-00000-of-00001.parquet'
dfp1 = pd.read_parquet(file_path)
print(dfp1.head())


                nome          cpf                     email  \
0       Agatha Gomes  47030856073    larissa95@yahoo.com.br   
1  Agatha Nascimento  53686156562  fogacaraquel@silveira.br   
2       Agatha Nunes  36537807266          gmelo@freitas.br   
3    Agatha Oliveira  65631925523        yasmin76@barros.br   
4     Agatha Ribeiro  71612587231    matheus75@monteiro.org   

              DT_CARGA  
0  2024-09-11 15:17:40  
1  2024-09-11 15:17:40  
2  2024-09-11 15:17:40  
3  2024-09-11 15:17:40  
4  2024-09-11 15:17:40  


In [44]:
# Lendo aqreuivo parquet 2.
file_path = 'output/ranking_clientes.parquet-00000-of-00001.parquet'
dfp2 = pd.read_parquet(file_path)
print(dfp2.head())


           cpf          numeroConta         numeroCartao ranking  \
0  18046079841  5100505060838687527  2097828104877643350           
1  81465703213  7595295926917893421  3442201103375110717       A   
2  58695342240  1762048877978329900  4818239517605922667           
3  78857172116  9117145463574074166  1626022023864896110           
4  80101594884  4890295503896255487  1801738842007609011           

              DT_CARGA  
0  2024-09-11 15:17:40  
1  2024-09-11 15:17:40  
2  2024-09-11 15:17:40  
3  2024-09-11 15:17:40  
4  2024-09-11 15:17:40  
